## 2.1 SPOTlight

In [ ]:
library(ggplot2)
library(SPOTlight)
library(SingleCellExperiment)
library(SpatialExperiment)
library(scater)
library(scran)
library(scuttle)
library(Matrix)
library(data.table)
library(Seurat)
library(dplyr)
library(gt)
library(igraph)
library(RColorBrewer)
library(scuttle)


process_all_subsets <- function(sce, conditions = c("Sol", "EDL", "LD"),
                                base_dirs = list(
                                  Sol = "Z:\\A\\ST\\Analysis\\Sol",
                                  EDL = "Z:\\A\\ST\\Analysis\\EDL",
                                  LD  = "Z:\\A\\ST\\Analysis\\LD"
                                ),
                                filename = "filtered_feature_bc_matrix.h5") {
  library(Seurat)
  library(SpatialExperiment)
  library(SingleCellExperiment)
  library(scater)

  sce_list <- lapply(conditions, function(cond) {
    subset(sce, subset = orig.ident == cond)
  })
  names(sce_list) <- conditions

  spe_list <- lapply(conditions, function(cond) {
    Load10X_Spatial(data.dir = base_dirs[[cond]], filename = filename)
  })
  names(spe_list) <- conditions
  
  for (i in seq_along(conditions)) {
    cond <- conditions[i]
    spe_list[[cond]]@meta.data$orig.ident <- cond
    spe_list[[cond]]@images <- spe_list[[cond]]@images[i] 
  }
  
  sce_sce_list <- lapply(sce_list, function(obj) {
    sce_obj <- as.SingleCellExperiment(obj)
    sce_obj <- logNormCounts(sce_obj)
    return(sce_obj)
  })
  
  return(list(
    sce_subsets = sce_sce_list,
    spe_subsets = spe_list
  ))
}

# result <- process_all_subsets(sce = sce)
# result$sce_subsets$Sol, result$sce_subsets$EDL, result$sce_subsets$LD 
# result$spe_subsets$Sol, result$spe_subsets$EDL, result$spe_subsets$LD 


genes <- !grepl(pattern = "^Rp[l|s]|Mt", x = rownames(sce.Sol))
dec <- modelGeneVar(sce.Sol, subset.row = genes)
plot(dec$mean, dec$total, xlab = "Mean log-expression", ylab = "Variance")
curve(metadata(dec)$trend(x), col = "blue", add = TRUE)

hvg <- getTopHVGs(dec, n = 3000)

colLabels(sce.Sol) <- colData(sce.Sol)$ident

mgs <- scoreMarkers(sce.Sol, subset.row = genes)
mgs_fil <- lapply(names(mgs), function(i) {
    x <- mgs[[i]]
    # Filter and keep relevant marker genes, those with AUC > 0.8
    x <- x[x$mean.AUC > 0.6, ]
    # Sort the genes from highest to lowest weight
    x <- x[order(x$mean.AUC, decreasing = TRUE), ]
    # Add gene and cluster id to the dataframe
    x$gene <- rownames(x)
    x$cluster <- i
    data.frame(x)
})
mgs_df <- do.call(rbind, mgs_fil)

idx <- split(seq(ncol(sce.Sol)), sce.Sol$ident)

n_cells <- 100
cs_keep <- lapply(idx, function(i) {
    n <- length(i)
    if (n < n_cells)
        n_cells <- n
    sample(i, n_cells)
})
sce.Sol <- sce.Sol[, unlist(cs_keep)]

res <- SPOTlight(
    x = sce.Sol,
    y = spe.Sol,
    groups = as.character(sce.Sol$ident),
    mgs = mgs_df,
    hvg = hvg,
    weight_id = "mean.AUC",
    group_id = "cluster",
    gene_id = "gene")

head(mat <- res$mat)[, seq_len(3)]
mod <- res$NMF

plotTopicProfiles(
    x = mod,
    y = sce.Sol$ident,
    facet = FALSE,
    min_prop = 0.01,
    ncol = 1) +
    theme(aspect.ratio = 1)

plotTopicProfiles(
    x = mod,
    y = sce.Sol$ident,
    facet = TRUE,
    min_prop = 0.01,
    ncol = 6)

plotCorrelationMatrix(mat)

plotInteractions(mat, which = "heatmap", metric = "prop")

plotInteractions(mat, which = "network")

ct <- colnames(mat)
mat[mat < 0.1] <- 0

paletteMartin <- c(
    "#000000", "#004949", "#009292", "#ff6db6", "#ffb6db", 
    "#490092", "#006ddb", "#b66dff", "#6db6ff", "#b6dbff", 
    "#920000", "#924900", "#db6d00", "#24ff24", "#ffff6d")

pal <- colorRampPalette(paletteMartin)(length(ct))
names(pal) <- ct

plotSpatialScatterpie(
    x = spe.Sol,
    y = mat,
    cell_types = colnames(mat),
    img = FALSE,
    scatterpie_alpha = 1,
    pie_scale = 0.4) +
    scale_fill_manual(
        values = pal,
        breaks = names(pal))


## 2.2 Moran's I

In [ ]:
library(Seurat)
library(spdep)
library(ape)

calculate_morans_i <- function(seurat_obj, gene_name, expression_threshold = 1) {
    if (!gene_name %in% rownames(seurat_obj[["Spatial"]])) {
        stop(paste(gene_name, "is not a valid feature in the Seurat object."))
    }

    gene_expression <- FetchData(seurat_obj, gene_name)

    if (is.null(gene_expression)) {
        stop("Gene expression data could not be retrieved.")
    }

    expressing_cells <- gene_expression[gene_expression[, gene_name] > expression_threshold, , drop = FALSE]

    if (nrow(expressing_cells) == 0) {
        stop(paste("No cells with", gene_name, "expression above", expression_threshold, "were found."))
    }

    coords <- GetTissueCoordinates(seurat_obj)
    if (is.null(coords)) {
        stop("Spatial coordinates data could not be retrieved.")
    }

    expressing_coords <- coords[rownames(expressing_cells), ]

    if (nrow(expressing_coords) != nrow(expressing_cells)) {
        stop("The number of expressing cells does not match the number of coordinates retrieved.")
    }

    distances <- as.matrix(dist(cbind(expressing_coords$imagerow, expressing_coords$imagecol)))

    distance_threshold <- quantile(distances, probs = 0.1, na.rm = TRUE)

    neighbors <- dnearneigh(cbind(expressing_coords$imagerow, expressing_coords$imagecol), 0, distance_threshold)

    weights <- nb2listw(neighbors, style = "W", zero.policy = TRUE)

    moran <- moran.test(expressing_cells[, gene_name], weights)

    return(moran)
}

moran_result <- calculate_morans_i(Sol, "PECAM1", 1)
print(moran_result)

In [ ]:
## 2.3 Ripley's K

In [ ]:
library(Seurat)
library(spatstat)
library(spatstat.geom)
library(spatstat.core)

generate_ripleys_k_plot <- function(seurat_obj, gene_name, expression_threshold) {
    if (!"slice1" %in% names(seurat_obj@images)) {
        stop("The Seurat object does not contain spatial coordinates information under the expected name 'slice1'.")
    }
    coords <- seurat_obj@images$slice1@coordinates
    
    expr <- FetchData(seurat_obj, gene_name)
    
    expressing_cells <- which(expr[[gene_name]] > expression_threshold)
    
    if (length(expressing_cells) > 0) {
        filtered_coords <- coords[expressing_cells, , drop = FALSE]
        
        if (nrow(filtered_coords) > 0) {
            x_range <- range(filtered_coords[, 'row'], na.rm = TRUE)
            y_range <- range(filtered_coords[, 'col'], na.rm = TRUE)
            
            if (x_range[1] < x_range[2] && y_range[1] < y_range[2]) {
                expr_points <- ppp(x = filtered_coords[, 'row'], y = filtered_coords[, 'col'],
                                   window = owin(xrange = x_range, yrange = y_range))
                
                K_result <- Kest(expr_points)
                
                plot(K_result, main = paste("Ripley's K Function for", gene_name, "Gene Expression"))
            } else {
                stop("Invalid ranges for x or y coordinates.")
            }
        } else {
            warning("No cells expressing", gene_name, "above the threshold of", expression_threshold)
        }
    } else {
        warning("No expressing cells found for gene", gene_name, "above the threshold of", expression_threshold)
    }
}

generate_ripleys_k_plot(, "", ) 
